# Analysis of distribution of shape statistics

**Part 3: plot cities**

This notebook visualizes the results of the peak-finding procedure from the previous notebook, `04a_peaks_compute`, applied to the geometries of the FUAs. 

In [ ]:
import os

os.environ["USE_PYGEOS"] = "0"
import json
import geopandas
import pandas
import numpy
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from palettable.cartocolors.qualitative import Bold_6

from scipy.signal import find_peaks
from scipy.stats import gaussian_kde

Set default plotting theme and add color dictionary for continents.

In [ ]:
sns.set_theme(
    context="paper",
    style="ticks",
    rc={
        "patch.force_edgecolor": False,
        "axes.spines.top": False,
        "axes.spines.right": False,
        "axes.grid": True,
    },
    palette=Bold_6.hex_colors,
)

coldict = {
    "Africa": 0,
    "Asia": 1,
    "Europe": 2,
    "North America": 3,
    "Oceania": 4,
    "South America": 5,
}

Load results from previous notebook

In [ ]:
results = json.load(open("../results/04_peaks_results.json"))

# load parameters (metric names)
options = [
    "circular_compactness_index",
    "isoperimetric_quotient_index",
    "isoareal_quotient_index",
    "radii_ratio_index",
    "diameter_ratio_index",
]

Load sample meta data

In [ ]:
sample = geopandas.read_parquet("../data/sample.parquet")

Plot settings and subfolders

In [ ]:
mydirs = [
    "../plots/cities/birdview",
    "..plots/cities/detail",
]

for mydir in mydirs:
    os.makedirs(mydir, exist_ok=True)

rcParams.update({"figure.autolayout": True})

Birdview plots of detected graph face artefacts, using circular compactness index threshold as metric

In [ ]:
option = "circular_compactness_index"

for city in results.keys():

    threshold = results[city][option]["threshold"]

    if threshold:
        # get block data for current city
        city_id = int(sample.loc[sample["eFUA_name"]==city, "eFUA_ID"].values)
        fua = geopandas.read_parquet(f"../data/{int(city_id)}/polygons/")
        
        # initialize figure
        fig, ax = plt.subplots(1, 1, figsize=(20, 20))

        # plot FUA boundary
        sample[sample["eFUA_name"]==city].to_crs(fua.crs).exterior.plot(
            ax=ax, linewidth = 2, color = "black", linestyle = "dotted")

        # # plot FUA interior
        # sample[sample["eFUA_name"]==city].to_crs(fua.crs).plot(
        #     ax=ax, color = "grey", alpha = 0.07)

        # plot true urban blocks
        fua[fua["circular_compactness_index"] > threshold].plot(
            ax=ax, color="grey", alpha=0.1
        )

        # plot street artifacts
        fua[fua["circular_compactness_index"] <= threshold].plot(
            ax=ax, color="red", alpha=1
        )

        ax.set_axis_off()

        plt.rc("figure", titlesize=50)
        fig.suptitle(city.capitalize())
        fig.savefig(
            f"../plots/cities/birdview/{city}.png", dpi=300, bbox_inches="tight"
        )
        plt.close()
        print(f"Plotted {city}")
    else:  # if no threshold has been found, try to estimate one
        print(f"No threshold found for {city}")